## PART A: Recurrent Neural Network & Classification: 

1. Data Processing: This data set is a bit messy, so the preprocessing portion is largely a tutorial to make sure students have data ready for keras. 

a) Import the following libraries: 

In [ ]:
import sys
import os
import json
import pandas
import numpy
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.layers import Embedding
from keras.preprocessing import sequence

#%pip install keras

Note: you may need to restart the kernel to use updated packages.


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from collections import OrderedDict

b) We will read the code in slightly differently than before: 

In [8]:
dataframe = pandas.read_csv("dev-access.csv", engine='python', quotechar='|', header=None)

c) We then need to convert to a numpy.ndarray type: 

In [9]:
dataset = dataframe.values

d) Check the shape of the data set - it should be (26773, 2). Spend some time looking at the data. 

In [12]:
print("Shape of the dataset: ", dataset.shape)
print("Number of rows: ", dataset.shape[0])
print("Number of columns: ", dataset.shape[1])

Shape of the dataset:  (26773, 2)
Number of rows:  26773
Number of columns:  2


e) Store all rows and the 0th index as the feature data: 

In [13]:
X = dataset[:,0]

f) Store all rows and index 1 as the target variable: 

In [14]:
Y = dataset[:,1]

g) In the next step, we will clean up the predictors. This includes removing features that are not valuable, such as timestamp and source. 

In [15]:
for index, item in enumerate(X):
    # Quick hack to space out json elements
    reqJson = json.loads(item, object_pairs_hook=OrderedDict)
    del reqJson['timestamp']
    del reqJson['headers']
    del reqJson['source']
    del reqJson['route']
    del reqJson['responsePayload']
    X[index] = json.dumps(reqJson, separators=(',', ':'))

h) We next will tokenize our data, which just means vectorizing our text. Given the data we will tokenize every character (thus char_level = True).

In [16]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)

i) Need to pad our data as each observation has a different length.

In [17]:
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)

j) Create your train set to be 75% of the data and your test set to be 25%.

In [21]:
train_size = int(len(X_processed) * 0.75)
X_train = X_processed[:train_size]
X_test = X_processed[train_size:]
Y_train = Y[:train_size]
Y_test = Y[train_size:]

print("Shape of the training set: ", X_train.shape)
print("Shape of the test set: ", X_test.shape)

Shape of the training set:  (20079, 1024)
Shape of the test set:  (6694, 1024)


## 2. Model 1 - RNN: The first model will be a pretty minimal RNN with only an embedding layer, simple RNN and Dense layer. The next model we will add a few more layers.

a) Start by creating an instance of a Sequential model: https://keras.io/getting-started/sequential-model-guide/Links to an external site.

b) From there, add an Embedding layer: https://keras.io/layers/embeddings/Links to an external site.

Params:
- input_dim = num_words (the variable we created above)
- output_dim = 32
- input_length = max_log_length (we also created this above)
- Keep all other variables as the defaults (shown below)

c) Add a SimpleRNN layer: https://keras.io/layers/recurrent/Links to an external site.

Params:
- units = 32
- activation = 'relu'

d) Finally, we will add a Dense layer: https://keras.io/layers/core/#denseLinks to an external site.

Params:
- units = 1 (this will be our output)
- activation --> you can choose to use either relu or sigmoid. 

e) Compile model using the .compile() method: https://keras.io/models/model/Links to an external site.

Params:
- loss = binary_crossentropy
- optimizer = adam
- metrics = accuracy

f) Print the model summary.

g) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

h) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

## 3) Model 2 - LSTM + Dropout Layers:

Now we will add a few new layers to our RNN and incorporate the more powerful LSTM. You will be creating a new model here, so make sure to call it something different than the model from Part 2.

a) This RNN needs to have the following layers (add in this order):

- Embedding Layer (use same params as before)
- LSTM Layer (units = 64, recurrent_dropout = 0.5)
- Dropout Layer - use a value of 0.5
- Dense Layer - (use same params as before)

b) Compile model using the .compile() method:

Params:
- loss = binary_crossentropy
- optimizer = adam
- metrics = accuracy

c) Print the model summary.

d) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

e) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

## 4) Recurrent Neural Net Model 3: Build Your Own

You will now create your RNN based on what you have learned from Model 1 & Model 2:

a) RNN Requirements:
- Use 5 or more layers
- Add a layer that was not utilized in Model 1 or Model 2 (Note: This could be a new Dense layer or an additional LSTM).

b) Compiler Requirements:
- Try a new optimizer for the compile step
- Keep accuracy as a metric (feel free to add more metrics if desired)

c) Print the model summary.

d) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

e) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

## Conceptual Questions: 

5) Explain the difference between the relu activation function and the sigmoid activation function.

_The ReLU activation function just outputs the input if it’s positive, and 0 if it’s negative. So it’s like:_
_ReLU(x) = max(0, x)_

_The sigmoid function squishes the input into a range between 0 and 1 using a curve that looks like an “S”:_
_Sigmoid(x) = 1 / (1 + e^(-x))_

_ The main differences are:_
_1. ReLU is super simple and fast, and it doesn’t squish big values down._
_2. Sigmoid is good when we need outputs between 0 and 1 like probabilities, but it can cause the vanishing gradient problem in deep networks, making learning slower._

6) Describe what one epoch actually is (epoch was a parameter used in the .fit() method).

_An epoch is one complete pass through the entire training dataset._

_For example, if I have 1,000 examples, and I run fit with 5 epochs, it means the model will see all 1.000 examples 5 times during training just in different shuffled orders each time._

7) Explain how dropout works (you can look at the keras code and/or documentation) for (a) training, and (b) test data sets.

_Dropout is a way to prevent overfitting by randomly turning off some neurons during training._

_During training dropout randomly sets some neurons outputs to 0. This forces the model to not depend too much on any one neuron and helps it generalize better._

_During testing the dropout is turned off and all neurons are used. But the output is scaled to make up for the fact that no neurons are dropped._

8) Explain why problems such as this homework assignment are better modeled with RNNs than CNNs. What type of problem will CNNs outperform RNNs on?

###......

_CNNs are better for images or spatial data, where local patterns like edges or shapes are more important than the order of the input. So CNNs will usually outperform RNNs on problems like image classification or object detection._

9) Explain what RNN problem is solved using LSTM and briefly describe how.

_RNNs can have trouble remembering things from far back in the sequence. This is called the vanishing gradient problem, it means the memory kind of fades as we go deeper into the sequence like in very large sentences._

_LSTMs fix this by adding a memory cell that can store information for a long time. It uses gates to decide:_

_- What to keep_

_- What to throw away_

_- And what new stuff to add_

_This helps the model remember important info from earlier in the sequence, which regular RNNs often forgets._